In [5]:
!pip install autocorrect

     |████████████████████████████████| 19.1MB 228kB/s 
  Created wheel for autocorrect: filename=autocorrect-0.4.4-cp36-none-any.whl size=19134813 sha256=7e5e63b434efd991ff10484fb9d0f8201d4ab6b256d55b31b7fa3f396006c06f
  Stored in directory: /root/.cache/pip/wheels/e5/80/6d/feed99c22f3f15b0b6881569dc9021f7e94b6464aa0688d819
Successfully built autocorrect


In [4]:
import nltk
nltk.download('popular')

[nltk_data] Downloading collection 'popular'
[nltk_data]    | 
[nltk_data]    | Downloading package cmudict to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/cmudict.zip.
[nltk_data]    | Downloading package gazetteers to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/gazetteers.zip.
[nltk_data]    | Downloading package genesis to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/genesis.zip.
[nltk_data]    | Downloading package gutenberg to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/gutenberg.zip.
[nltk_data]    | Downloading package inaugural to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/inaugural.zip.
[nltk_data]    | Downloading package movie_reviews to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping corpora/movie_reviews.zip.
[nltk_data]    | Downloading package names to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/names.zip.
[nltk_data]    | Downloading package shakespeare to /root/nltk_data...
[nlt

True

In [0]:
import random
import collections
import pandas as pd
import numpy as np
import re
from nltk.corpus import stopwords
from autocorrect import Speller
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer 
from nltk.metrics import *

# Init the Wordnet Lemmatizer
lemmatizer = WordNetLemmatizer()



In [1]:
from google.colab import files
import io


uploaded = files.upload()

Saving music_intent_entities.json to music_intent_entities.json
Saving restaurant_intent_entities.json to restaurant_intent_entities.json
Saving weather_intent_entities.json to weather_intent_entities.json


In [0]:

music_file = pd.read_json(io.BytesIO(uploaded['music_intent_entities.json']))
restaurant_file = pd.read_json(io.BytesIO(uploaded['restaurant_intent_entities.json']))
weather_file = pd.read_json(io.BytesIO(uploaded['weather_intent_entities.json']))

In [0]:
data = {};
data['music'] = music_file['text'].to_numpy()
data['restaurant'] = restaurant_file['text'].to_numpy()
data['weather'] = weather_file['text'].to_numpy()


Getting the words from the data

In [0]:
all_words = []


document = [(text, category) for category in data.keys() for text in data[category]]
random.shuffle(document)

array_words = [nltk.word_tokenize(w) for (w, cat) in document];
flat_list = [word for sent in array_words for word in sent]

Removes the **stop words** like ( ‘off’, ‘is’, ‘s’, ‘am’, ‘or’) and  ***non alphabetical*** characters

In [0]:
stopWords = set(stopwords.words('english'))

def remove_stop_words(words):
  wordsFiltered = []

  for w in words:
    if w not in stopWords:
      if w.isalpha():
        wordsFiltered.append(w)

  return wordsFiltered

flat_list=remove_stop_words(flat_list)

**Lemmatization** i.e., tranforms tarnsforms different forms of words to a single one

In [0]:
def get_wordnet_pos(word):
    """Map POS tag to first character lemmatize() accepts"""
    tag = nltk.pos_tag([word])[0][1][0].upper()
    tag_dict = {"J": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}

    return tag_dict.get(tag, wordnet.NOUN)


def lemmatization(words):
  return [lemmatizer.lemmatize(w, get_wordnet_pos(w)) for w in words]


filtered_list=lemmatization(flat_list)

Getting the ***frequency*** of each word and extracting top 2000

In [0]:

frequencyDistribution = nltk.FreqDist(w.lower() for w in filtered_list)
word_features = list(frequencyDistribution)[:2000]


**FEATURE** **EXTRACTION**

In [0]:

def feature_Extraction(doc):
    document_words = [word.lower() for word in nltk.word_tokenize(doc)]
    
    document_words=remove_stop_words(document_words)
    document_words=lemmatization(document_words)
    features = {}
    for word in word_features:
        features['contains({})'.format(word)] = (word in document_words)
    return features

Training the model

In [0]:
test_set=nltk.classify.apply_features(feature_Extraction, document[:500])
train_set=nltk.classify.apply_features(feature_Extraction, document[500:])
classifier = nltk.NaiveBayesClassifier.train(train_set)

Testing the model *accuracy*

In [15]:
print(nltk.classify.accuracy(classifier, test_set))

0.996


In [16]:
classifier.show_most_informative_features(20)

Most Informative Features
         contains(table) = True           restau : weathe =    329.0 : 1.0
          contains(book) = True           restau : music  =    318.6 : 1.0
    contains(restaurant) = True           restau : weathe =    123.2 : 1.0
        contains(people) = True           restau : music  =     77.4 : 1.0
           contains(eat) = True           restau : weathe =     71.7 : 1.0
          contains(food) = True           restau : weathe =     67.7 : 1.0
   contains(reservation) = True           restau : weathe =     57.5 : 1.0
      contains(national) = True           weathe : music  =     52.6 : 1.0
           contains(rat) = True           restau : music  =     51.3 : 1.0
          contains(find) = True           restau : weathe =     47.0 : 1.0
        contains(minute) = True           weathe : music  =     42.8 : 1.0
          contains(week) = True           weathe : music  =     42.1 : 1.0
          contains(make) = True           restau : weathe =     41.0 : 1.0

**OUTPUTS**

In [17]:
print(classifier.classify(feature_Extraction("Is it sunnier today?")))
print(classifier.classify(feature_Extraction("book a table")))
print(classifier.classify(feature_Extraction(" I want to listen to popular telugu song ")))


weather
restaurant
music


Measuring **Precision,Recall,F-Measure** of a classifier.Finding **Confusion matrix**


In [0]:
actual_set  = collections.defaultdict(set)
predicted_set  = collections.defaultdict(set)
# cm here refers to confusion matrix
actual_set_cm = []
predicted_set_cm = []

In [0]:
for i, (feature, label) in enumerate(test_set):
    actual_set[label].add(i)
    actual_set_cm.append(label)
    predicted_label = classifier.classify(feature)
    predicted_set[predicted_label].add(i)
    predicted_set_cm.append(predicted_label)


In [41]:
for category in data.keys():
  print(category,'precision :',precision(actual_set[category], predicted_set[category]))
  print(category,'recall :',recall(actual_set[category], predicted_set[category]))
  print(category,'f-measure :',f_measure(actual_set[category], predicted_set[category]))

music precision : 1.0
music recall : 1.0
music f-measure : 1.0
restaurant precision : 0.9935897435897436
restaurant recall : 0.9935897435897436
restaurant f-measure : 0.9935897435897437
weather precision : 0.994535519125683
weather recall : 0.994535519125683
weather f-measure : 0.994535519125683


In [43]:
confusion_matrix = ConfusionMatrix(actual_set_cm, predicted_set_cm)
print (confusion_matrix)

           |       r     |
           |       e     |
           |       s     |
           |       t   w |
           |       a   e |
           |   m   u   a |
           |   u   r   t |
           |   s   a   h |
           |   i   n   e |
           |   c   t   r |
-----------+-------------+
     music |<161>  .   . |
restaurant |   .<155>  1 |
   weather |   .   1<182>|
-----------+-------------+
(row = reference; col = test)

